In [1]:
!wget http://www.grouplens.org/system/files/ml-100k.zip

--2017-04-17 17:25:06--  http://www.grouplens.org/system/files/ml-100k.zip
Resolving www.grouplens.org... 128.101.34.146
Connecting to www.grouplens.org|128.101.34.146|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://grouplens.org/system/files/ml-100k.zip [following]
--2017-04-17 17:25:07--  https://grouplens.org/system/files/ml-100k.zip
Resolving grouplens.org... 128.101.34.146
Connecting to grouplens.org|128.101.34.146|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://files.grouplens.org/papers/ml-100k.zip [following]
--2017-04-17 17:25:08--  http://files.grouplens.org/papers/ml-100k.zip
Resolving files.grouplens.org... 128.101.34.146
Connecting to files.grouplens.org|128.101.34.146|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [application/zip]
Saving to: 'ml-100k.zip'

ml-100k.zip         100%[===================>]   4.70M  1.14MB/s    in 5.5s  

In [2]:
!unzip ml-100k.zip

Archive:  ml-100k.zip
   creating: ml-100k/
  inflating: ml-100k/allbut.pl       
  inflating: ml-100k/mku.sh          
  inflating: ml-100k/README          
  inflating: ml-100k/u.data          
  inflating: ml-100k/u.genre         
  inflating: ml-100k/u.info          
  inflating: ml-100k/u.item          
  inflating: ml-100k/u.occupation    
  inflating: ml-100k/u.user          
  inflating: ml-100k/u1.base         
  inflating: ml-100k/u1.test         
  inflating: ml-100k/u2.base         
  inflating: ml-100k/u2.test         
  inflating: ml-100k/u3.base         
  inflating: ml-100k/u3.test         
  inflating: ml-100k/u4.base         
  inflating: ml-100k/u4.test         
  inflating: ml-100k/u5.base         
  inflating: ml-100k/u5.test         
  inflating: ml-100k/ua.base         
  inflating: ml-100k/ua.test         
  inflating: ml-100k/ub.base         
  inflating: ml-100k/ub.test         


In [53]:
import numpy
import pandas
import cPickle as pickle
from sklearn.metrics import roc_auc_score, mean_squared_error

In [54]:
from fastFM.mcmc import FMClassification, FMRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.datasets import dump_svmlight_file

In [71]:
!cat ml-100k/README

SUMMARY & USAGE LICENSE

MovieLens data sets were collected by the GroupLens Research Project
at the University of Minnesota.
 
This data set consists of:
	* 100,000 ratings (1-5) from 943 users on 1682 movies. 
	* Each user has rated at least 20 movies. 
        * Simple demographic info for the users (age, gender, occupation, zip)

The data was collected through the MovieLens web site
(movielens.umn.edu) during the seven-month period from September 19th, 
1997 through April 22nd, 1998. This data has been cleaned up - users
who had less than 20 ratings or did not have complete demographic
information were removed from this data set. Detailed descriptions of
the data file can be found at the end of this file.

Neither the University of Minnesota nor any of the researchers
involved can guarantee the correctness of the data, its suitability
for any particular purpose, or the validity of results based on the
use of the data set.  The data set may be used for any research
purposes under th

In [75]:
df = pd.read_csv("ml-100k/u.data",sep="\t", header=None, names=['user','item','rating','timestamp'])

In [79]:
X = df[['user','item']]
X.head()

,user,item
0,196,242
1,186,302
2,22,377
3,244,51
4,166,346


In [82]:
Y = df[['rating']]
Y.head()

,rating
0,3
1,3
2,1
3,2
4,1


In [108]:
from sklearn.model_selection import train_test_split

In [109]:
trainX, testX, trainY, testY = train_test_split(X, Y)


In [110]:
trainX.head()

,user,item
64355,648,104
88896,830,732
57871,839,257
9848,302,307
20340,157,476


In [111]:

encoder = OneHotEncoder(handle_unknown='ignore').fit(trainX)
trainX = encoder.transform(trainX)
testX = encoder.transform(testX)
rank = 10
n_iter=100
clf = FMRegression(rank=rank, n_iter=n_iter)


In [112]:
clf.predict(testX)

AttributeError: 'FMRegression' object has no attribute 'w_'

In [66]:
pd.read_csv("ml-100k/u.user",sep="|", header=None).head()

,0,1,2,3,4
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [4]:
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from pyfm import pylibfm

# Read in data
def loadData(filename,path="ml-100k/"):
    data = []
    y = []
    users=set()
    items=set()
    with open(path+filename) as f:
        for line in f:
            (user,movieid,rating,ts)=line.split('\t')
            data.append({ "user_id": str(user), "movie_id": str(movieid)})
            y.append(float(rating))
            users.add(user)
            items.add(movieid)

    return (data, np.array(y), users, items)

(train_data, y_train, train_users, train_items) = loadData("ua.base")
(test_data, y_test, test_users, test_items) = loadData("ua.test")
v = DictVectorizer()

/Users/nipunbatra/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [7]:
len(y_train)

90570

In [17]:
X_train = v.fit_transform(train_data)
X_test = v.transform(test_data)

In [20]:
import pandas as pd

In [23]:
fm = pylibfm.FM(num_factors=2, num_iter=5, verbose=True, task="regression", initial_learning_rate=0.001, learning_rate_schedule="optimal")

fm.fit(X_train,y_train)

Creating validation dataset of 0.01 of training for adaptive regularization
-- Epoch 1
Training MSE: 0.59437
-- Epoch 2
Training MSE: 0.51783
-- Epoch 3
Training MSE: 0.49048
-- Epoch 4
Training MSE: 0.47500
-- Epoch 5
Training MSE: 0.46475


In [25]:
preds = fm.predict(X_test)

In [27]:
X_test

<9430x2623 sparse matrix of type '<type 'numpy.float64'>'
	with 18858 stored elements in Compressed Sparse Row format>

In [35]:
!head -n 10 ml-100k/u.user

1|24|M|technician|85711
2|53|F|other|94043
3|23|M|writer|32067
4|24|M|technician|43537
5|33|F|other|15213
6|42|M|executive|98101
7|57|M|administrator|91344
8|36|M|administrator|05201
9|29|M|student|01002
10|53|M|lawyer|90703


In [47]:
from pyfm import pylibfm
from sklearn.feature_extraction import DictVectorizer
import numpy as np
train = [
    {"user": "1", "item": "5"},
    {"user": "2", "item": "43", "age": 33},
    {"user": "3", "item": "20", "age": 55},
    {"user": "4", "item": "10", "age": 20},
]
v = DictVectorizer()
X = v.fit_transform(train)

In [48]:
X.toarray()

array([[  0.,   0.,   0.,   0.,   1.,   1.,   0.,   0.,   0.],
       [ 33.,   0.,   0.,   1.,   0.,   0.,   1.,   0.,   0.],
       [ 55.,   0.,   1.,   0.,   0.,   0.,   0.,   1.,   0.],
       [ 20.,   1.,   0.,   0.,   0.,   0.,   0.,   0.,   1.]])

In [50]:
y = np.repeat(1.0,X.shape[0])
fm = pylibfm.FM()
fm.fit(X,y)
fm.predict(v.transform({"user": "1", "item": "10","age":20}))

Creating validation dataset of 0.01 of training for adaptive regularization
-- Epoch 1
Training log loss: 0.60801


array([ 0.99440535])

In [67]:
from fastFM.datasets import make_user_item_regression
from sklearn.cross_validation import train_test_split

# This sets up a small test dataset.
X, y, _ = make_user_item_regression(label_stdev=.4)

In [69]:
X.toarray()

array([[ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  1.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  1.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.]])

In [70]:
y

array([ 197.90847823,  206.12424048,  208.23634561,  207.86486241,
        205.25511467,  203.88078549,  194.0766025 ,  199.75929735,
        196.43007773,  202.02099645,  202.87595378,  203.04960735,
        200.19886833,  199.03235629,  207.44896091,  195.36254324,
        201.58224212,  207.26273533,  188.50250791,  205.44616174,
        212.13984042,  221.44992424,  223.29544218,  222.26993783,
        219.99592094,  218.34764696,  208.27667468,  213.90966806,
        210.68411163,  216.99290215,  218.25102917,  217.29163658,
        213.8052829 ,  212.77883467,  222.2459836 ,  209.73401168,
        215.02768204,  222.29569866,  202.01404443,  220.48050177,
        212.31046774,  220.85530445,  222.79199197,  222.44317089,
        219.74878333,  218.66456567,  207.34664573,  213.70152414,
        210.6745623 ,  216.50996108,  217.7169081 ,  217.74733784,
        214.2548381 ,  212.78936481,  222.79745549,  209.66787401,
        214.92888265,  221.0987156 ,  202.06347151,  220.64864